In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
    .appName("sub_product_price") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/14 16:44:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .format('iceberg') \
    .load('iceberg.bronze.product_details')
# df.printSchema()
# df.show()

In [4]:
df = df.select(
    col('id').alias('product_id'),
    'price',
    'original_price',
    'discount',
    'discount_rate',
    'ngay_cap_nhat'
).withColumn("ngay_cap_nhat_date", F.to_date('ngay_cap_nhat'))
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- original_price: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- discount_rate: string (nullable = true)
 |-- ngay_cap_nhat: timestamp (nullable = true)
 |-- ngay_cap_nhat_date: date (nullable = true)



In [5]:
max_date = df.agg(F.max("ngay_cap_nhat_date")).first()[0]

df = df.filter(col('ngay_cap_nhat_date') == max_date)
df = df.drop('ngay_cap_nhat_date', 'ngay_cap_nhat')

In [6]:
df = df.dropDuplicates()

In [7]:
df = df.withColumn("price", col("price").cast("long")) \
    .withColumn("original_price", col("original_price").cast("long")) \
    .withColumn("discount", col("discount").cast("long")) \
    .withColumn("discount_rate", col("discount_rate").cast("float"))

In [8]:
df = df.filter(col("product_id").isNotNull())

In [9]:
df = df.withColumn("ngay_cap_nhat", current_timestamp())

In [10]:
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- price: long (nullable = true)
 |-- original_price: long (nullable = true)
 |-- discount: long (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- ngay_cap_nhat: timestamp (nullable = false)



In [11]:
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.silver.sub_product_price')

In [12]:
spark.stop()